In [ ]:
# import time
# import psutil
# from opcua import ua,Server, uamethod

# namespace = "PC_Data"
# server_url = "opc.tcp://127.0.0.1:12348"

# server =Server()

# ## Method which we want to publish over the OPCUA
# @uamethod
# def multiply(parent, x:float , y:float):
#     print(f"Multiply method call with parameters {x, y}")
#     return x * y


# server.set_endpoint(server_url)
# idx = server.register_namespace(namespace)

# objects = server.get_objects_node()
# my_function =  objects.add_object(idx ,  "MyFunction")

# x, x.Name , x.DataType =  ua.Argument(), "x" , ua.NodeId(ua.ObjectIds.Int64)
# y, y.Name , y.DataType =  ua.Argument(), "y" , ua.NodeId(ua.ObjectIds.Int64)
# result , result.Name , result.DataType =  ua.Argument(), "result" , ua.NodeId(ua.ObjectIds.Int64)


# my_function.add_method(idx, "multiply" ,  multiply, [x,y], [result])


# try:
#     server.start()
#     print("Multiply OPC UA server started")

#     while True:
#         time.sleep(3)

# finally:
#     server.stop()
#     print("Multiply OPC UA server stopped")




#######################################################
## Making a dummy plot for connecting into the server
#######################################################

## Assuing the data limits are not known

# import time
# import random
# from opcua import Server, ua
# import logging
# from opcua.crypto import security_policies
# from OpenSSL.crypto import load_privatekey, FILETYPE_PEM


# USER_CREDENTIALS = {
#     "XXXXXXXX": "XXXXXXXX"
# }

# # Authentification for the server with the username and password
# def user_authentication(username, password):
#     if username in USER_CREDENTIALS and USER_CREDENTIALS[username] == password:
#         return True
#     return False

# class SubHandler(object):
#     def datachangeNotification(self, node, val, data):
#         self.State_Code = val
#         print(f"Data change notification: Node={node} , value={val}")

#     def event_notification(self, event):
#         print(f"Event Notification: {event}")

#     def status_change_notification(self, status):
#         print(f"The status has been changed: {status}")

# # Initialize the OPC UA server
# server = Server()

# # Set endpoint
# server.set_endpoint("opc.tcp://127.0.0.1:12348")

# # Set namespace
# namespace = server.register_namespace("Auth")
# print(f"Namespace index {namespace}")
# logging.info(f"Namespace index: {namespace}")

# # Setup security policies and user manager
# policyIDs = ["Anonymous", "Basic256Sha256", "Username"]

# # Use the security policy for encryption and authentication
# server.set_security_IDs(policyIDs)
# server.set_security_policy([security_policies.SecurityPolicyBasic256Sha256])  # Encryption and security
# server.user_manager.set_user_manager(user_authentication)

# # Get the objects node
# objects = server.get_objects_node()
# server._setup_server_nodes()

# # Add a RoomTemperature variable
# room_temperature = objects.add_variable(namespace, "RoomTemperature", 22.5)
# room_temperature.set_writable()


# # Example of a password-protected variable
# sensitive_variable = objects.add_variable(namespace, "SensitiveData", "secret_value")
# sensitive_variable.set_writable()  # Allow write, but with authentication

# # Start the server with authentication and encryption
# try:
#     server.start()
#     print("===================== Server is online")


#     # Create subscription for data change notifications
#     handler = SubHandler()
#     subscription = server.create_subscription(10, handler)

#     # Monitoring the temperature
#     monitor = subscription.subscribe_data_change(room_temperature)

#     while True:
#         # Simulate random temperature changes
#         temp = room_temperature.get_value() + random.uniform(-1, 1)
#         room_temperature.set_value(temp)


#         print(f"The current temperature is: {room_temperature.get_value()}")

#         # Simulate updating sensitive data, allowing write access based on credentials
#         if user_authentication("XXXXXXXX", "XXXXXXXX"):
#             sensitive_variable.set_value("new_secret_value")
#             print(f"Sensitive data updated: {sensitive_variable.get_value()}")
#         else:
#             print("Unauthorized access attempt to sensitive data")

#         time.sleep(2)
# except Exception as e:
#     print(f"Server failed to start: {e}")




########################################################################
## Trying the code with the encryption --> Working code
########################################################################



# import sys
# import time
# from opcua.server.user_manager import UserManager
# from opcua.common.xmlexporter import XmlExporter

# sys.path.insert(0, "..")

# from opcua import ua, Server

# ## User credentials for logging in
# USER_CREDENTIALS = {
#     "XXXXXXXX": "XXXXXXXX"
# }

# def user_authentication(isession, username, password):
#     isession.user =  UserManager.User
#     if username in USER_CREDENTIALS and USER_CREDENTIALS[username]== password:
#         return True
#     return False

# if __name__ == "__main__":

#     # setup our server
#     server = Server()
#     server.set_endpoint("opc.tcp://127.0.0.1:12348")

#     ## Setting the server name
#     servername = "My_Server"
#     server.set_server_name(servername)

#     # server.set_security_policy([ua.SecurityPolicyType.Basic256Sha256_SignAndEncrypt])
#     # load server certificate and private key. This enables endpoints
#     # with signing and encryption.
#     # server.load_certificate(r"C:\Users\Jeevan.Somanna\Downloads\certificate-example.der")
#     # server.load_private_key(r"C:\Users\Jeevan.Somanna\Downloads\private-key-example.pem")

#     ## Add the security policy ids
#     # policy_ids =  ['Username']
#     server.set_security_IDs(['Username'])
#     server.allow_anonymous = False
#     server.user_manager.set_user_manager(user_authentication)
#     # server.user_manager= user_authentication

#     # setup our own namespace, not really necessary but should as spec
#     uri = "My_Own_Namespace"
#     idx = server.register_namespace(uri)

#     # get Objects node, this is where we should put our custom stuff
#     objects = server.get_objects_node()

#     # populating our address space
#     myobj = objects.add_object(idx, "MyObject")
#     myvar = myobj.add_variable(idx, "MyVariable", 6.7)
#     myvar.set_writable()    # Set MyVariable to be writable by clients

#     # starting!

#     try:
#         server.export_xml_by_ns("Server_model.xml" , namespaces="My_Own_Namespace")
#         server.start()
#         count = 0
#         while True:
#             time.sleep(1)
#             count += 0.1
#             myvar.set_value(count)
#     finally:
#         #close connection, remove subcsriptions, etc
#         server.stop()




##############################################################################
##*****Alternative Method for the verifying with the username and the password****
##############################################################################


##############################################################################################
#### Creating a new systems for the verifying the security of the server
##############################################################################################

import socket
import bcrypt
import json
from opcua import Server
import time
from random import randint

class OPC_Server_Security:

    def __init__(self):
        self.server_socket = socket.socket()
        self.salt = bcrypt.gensalt()

    def init_opc_server_security(self, ip):
        self.server_socket.bind((ip, 12350))
        self.server_socket.listen(1)

    def set_server_credentials(self, username, password):
        pwd = password.encode('utf-8')
        user = username.encode('utf-8')
        server_hash_pwd = bcrypt.hashpw(pwd, self.salt)
        server_hash_user = bcrypt.hashpw(user, self.salt)

        out_json = dict()

        out_json['username'] = server_hash_user.decode()
        out_json['password'] = server_hash_pwd.decode()
        out_json['salt'] = self.salt.decode()

        with open('credentials.json', 'w') as fp:
            json.dump(out_json, fp, indent=4, ensure_ascii=False)

    def client_authentication(self):
        config_file = open('credentials.json')
        config_data = json.load(config_file)
        config_file.close()

        is_client_authenticated = False

        print("Waiting for clients....")
        conn, address = self.server_socket.accept()
        while True:
            recvd_data = conn.recv(1024).decode()
            recvd_data = recvd_data.split(",")
            user = recvd_data[0]
            pwd = recvd_data[1]
            password = pwd.encode('utf-8')
            username = user.encode('utf-8')
            saved_salt = config_data['salt'].encode('utf-8')
            hashed_pwd = bcrypt.hashpw(password, saved_salt).decode()
            hashed_user = bcrypt.hashpw(username, saved_salt).decode()

            if hashed_user == config_data['username'] and hashed_pwd == config_data['password']:
                conn.send("Success".encode())
                is_client_authenticated = True
                break
            else:
                conn.send("Failure".encode())
                is_client_authenticated = False
                break

        if is_client_authenticated:
            print("Server access granted")
            return True
        else:
            print("Server access denied")
            return False



opc_server = OPC_Server_Security()
server = Server()

url = "opc.tcp://127.0.0.1:12349"

server.set_endpoint(url)

name = "OPC UA SIMULATION SERVER"
add_space = server.register_namespace(name)

node = server.get_objects_node()

param = node.add_object(add_space, "PARAMETERS")

temp = param.add_variable(add_space, "Temperature", 0)

temp.set_writable()
server.start()
print("Server started at {}".format(url))

opc_server.init_opc_server_security('127.0.0.1')
opc_server.set_server_credentials('admin', 'qwerty@123')

while True:
    res = opc_server.client_authentication()
    while res:
        temperature = randint(10, 50)
        temp.set_value(temperature)

    time.sleep(1)